# gpt generated -> film executable

In [ ]:
import os, json
sp = 'tests_seen'
path = f'turbo-bias-{sp}_result.json'
result = json.load(open(path))
for k in result.keys():    
    result[k]['triplet'][0] = result[k]['triplet'][0][1:]
    
os.getcwd()

In [ ]:
len(result.keys())

In [ ]:
for k in result.keys():    
    # print(result[k]['triplet'][0].split(','))
    tmp = []
    print(k)
    for i in range(len(result[k]['triplet'][0].split(','))):
        if 'Object' in result[k]['triplet'][0].split(',')[i]:
            action = result[k]['triplet'][0].split(',')[i].strip()
            obj = result[k]['triplet'][0].split(',')[i+1].strip()
            try:
                recep = result[k]['triplet'][0].split(',')[i+2].strip()
            except:
                recep = "0"
            tmp.append([action, obj, recep])
        # tmp.append([result[k]['triplet'][0].split(',')[3*i].strip(), result[k]['triplet'][0].split(',')[3*i + 1].strip(), result[k]['triplet'][0].split(',')[3*i + 2].strip()])
    result[k]['triplet_beforeDC'] = tmp
    result[k]['triplet'] = tmp
# with open(path, 'w') as f:
#     json.dump(result, f, indent=4)


## # Allowed actions: ToggleObject, CleanObject, HeatObject, PickupObject, SliceObject, CoolObject, PutObject, 0


In [ ]:
OPENABLE = ['Fridge', 'Cabinet', 'Microwave', 'Drawer', 'Safe']
ACTIONS4CHECK = ['CleanObject', 'HeatObject', 'CoolObject']
DC_effect = 0
fucked_keys = set()
for k in result.keys():    
    tmp_actions = []
    tmp_classes = []
    tmp_idxs = []
    skipped_cnt=0
    DC_indices = []
    for i in range(len(result[k]['triplet_beforeDC'])):
        action = result[k]['triplet_beforeDC'][i][0]
        target = result[k]['triplet_beforeDC'][i][1]
        recp = result[k]['triplet_beforeDC'][i][2]
        if action == 'HeatObject':
            tmp_actions.append('OpenObject')
            tmp_actions.append('PutObject')
            tmp_actions.append('CloseObject')
            tmp_actions.append('ToggleObjectOn')
            tmp_actions.append('ToggleObjectOff')
            tmp_actions.append('OpenObject')
            tmp_actions.append('PickupObject')
            tmp_actions.append('CloseObject')
            
            tmp_classes.append(recp) #Microwave
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(target)
            tmp_classes.append(recp)
            for _ in range(8):
                tmp_idxs.append(i-skipped_cnt)

            
        elif action == 'CoolObject':
            tmp_actions.append('OpenObject')
            tmp_actions.append('PutObject')
            tmp_actions.append('CloseObject')
            tmp_actions.append('OpenObject')
            tmp_actions.append('PickupObject')
            tmp_actions.append('CloseObject')
            
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(target)
            tmp_classes.append(recp)
            for _ in range(6):
                tmp_idxs.append(i-skipped_cnt)


        
        elif action == 'CleanObject':
            tmp_actions.append('PutObject')
            tmp_actions.append('ToggleObjectOn')
            tmp_actions.append('ToggleObjectOff')
            tmp_actions.append('PickupObject')
            
            tmp_classes.append('Sink')
            tmp_classes.append(recp)
            tmp_classes.append(recp)
            tmp_classes.append(target)
            for _ in range(4):
                tmp_idxs.append(i-skipped_cnt)

        elif action == 'ToggleObject':
            tmp_actions.append('ToggleObjectOn')
            
            tmp_classes.append(target)
            for _ in range(1):
                tmp_idxs.append(i-skipped_cnt)
                
        elif action == 'PickupObject':
            if recp in OPENABLE:
                tmp_actions.append('OpenObject')            
                tmp_actions.append('PickupObject')
                tmp_actions.append('CloseObject')            
                tmp_classes.append(recp)
                tmp_classes.append(target)
                tmp_classes.append(recp)
                for _ in range(3):
                    tmp_idxs.append(i-skipped_cnt)
            
            elif target == 'Laptop':  #Laptop  은 close 하고 pickup 해야한다
                tmp_actions.append('CloseObject')
                tmp_actions.append('PickupObject')
                tmp_classes.append(target)
                tmp_classes.append(target)
                for _ in range(2):
                    tmp_idxs.append(i-skipped_cnt)

            else:
                tmp_actions.append('PickupObject')
                tmp_classes.append(target)
                for _ in range(1):
                    tmp_idxs.append(i-skipped_cnt)

        
        elif action == 'SliceObject':
            tmp_actions.append('SliceObject')
            
            tmp_classes.append(target)
            for _ in range(1):
                tmp_idxs.append(i-skipped_cnt)
                
        elif action == 'PutObject':
            if recp in OPENABLE:
                tmp_actions.append('OpenObject')            
                tmp_actions.append('PutObject')
                tmp_actions.append('CloseObject')            
                tmp_classes.append(recp)
                tmp_classes.append(recp)
                tmp_classes.append(recp)
                for _ in range(3):
                    tmp_idxs.append(i-skipped_cnt)
            else:
                tmp_actions.append('PutObject')
                tmp_classes.append(recp)
                for _ in range(1):
                    tmp_idxs.append(i-skipped_cnt)
        else:
            print(action)
            skipped_cnt+=1
            fucked_keys.add(k)
        result[k]['low_actions'] = tmp_actions
        result[k]['low_classes'] = tmp_classes
        result[k]['high_idxs'] = tmp_idxs
    for j in reversed(DC_indices):
        del result[k]['triplet'][j]

print(len(fucked_keys))
print(DC_effect)


In [ ]:

with open(f'turbo-bias-{sp}_result_template_executable.json', 'w') as f:
    json.dump(result, f, indent=4)


In [ ]:
cnt = 0
for k in result.keys():
    if (len(result[k]['triplet'])) == 0:
        print(k)
        cnt+=1
print(cnt)

In [ ]:
cnt